In [140]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import pandas as pd
import scipy.stats

In [115]:
train_data = pd.read_csv('./mnist_train.csv', sep=',', header=None)
train_labels = train_data[0]
train_data = train_data.drop(0, axis=1)

test_data = pd.read_csv('./mnist_test.csv', sep=',', header=None)
test_labels = test_data[0]
test_data = test_data.drop(0, axis=1)

train_data = torch.FloatTensor(train_data.to_numpy())
train_labels = torch.LongTensor(train_labels.to_numpy())
test_data = torch.FloatTensor(test_data.to_numpy())

In [116]:
input_size = 784
output_size = 10
hidden_size = 50

epochs = 40
batch_size = 200
learning_rate = 0.00005

In [117]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [118]:
my_net = Net()

In [119]:
optimizer = torch.optim.SGD(my_net.parameters(), lr=learning_rate, momentum=0.9)
loss = nn.CrossEntropyLoss()

In [120]:
for i in range(epochs):
    for j in range(0, train_data.shape[0], batch_size):
        data_minibatch = Variable(train_data[j : j+batch_size])
        label_minibatch = Variable(train_labels[j: j+batch_size])
        optimizer.zero_grad()
        net_out = my_net(data_minibatch)
        net_loss = loss(net_out, label_minibatch)
        net_loss.backward()
        optimizer.step()

C:\Users\ethan\AppData\Local\Temp\ipykernel_4144\966333629.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [124]:
test_out = torch.max(net_out.data, 1)[1].numpy()

print(np.count_nonzero(test_out==test_labels) / len(test_labels))

0.955


In [121]:
activations = []

def get_activation():
    def hook(model, input, output):
        activations.append(output.detach())
    return hook

In [122]:
my_net.fc1.register_forward_hook(get_activation())
my_net.fc2.register_forward_hook(get_activation())
my_net.fc3.register_forward_hook(get_activation())
net_out = my_net(test_data)

C:\Users\ethan\AppData\Local\Temp\ipykernel_4144\966333629.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [129]:
activations[0] = ((abs(activations[0]) + activations[0])/2).numpy()
activations[1] = ((abs(activations[1]) + activations[1])/2).numpy()
activations[2] = F.log_softmax(activations[2]).numpy()

C:\Users\ethan\AppData\Local\Temp\ipykernel_4144\1366388366.py:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  activations[2] = F.log_softmax(activations[2]).numpy()


In [142]:
neurons = np.concatenate((activations[0].T, activations[1].T, activations[2].T))
n = len(neurons)
adj_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i != j:
            adj_matrix[i][j] = scipy.stats.pearsonr(neurons[i], neurons[j])[0]

In [145]:
print(adj_matrix.shape)
print(adj_matrix)

(110, 110)
[[ 0.          0.00201713 -0.07773352 ... -0.3024321  -0.16466779
  -0.38747545]
 [ 0.00201713  0.         -0.16207566 ...  0.25225757  0.23794172
   0.06290211]
 [-0.07773352 -0.16207566  0.         ... -0.32108208 -0.23415565
  -0.17908076]
 ...
 [-0.3024321   0.25225757 -0.32108208 ...  0.          0.20400336
   0.62481802]
 [-0.16466779  0.23794172 -0.23415565 ...  0.20400336  0.
   0.40640022]
 [-0.38747545  0.06290211 -0.17908076 ...  0.62481802  0.40640022
   0.        ]]


In [127]:
test_comp = np.matmul(my_net.fc1.weight.data.numpy(), test_data.numpy()[0])
test_comp = test_comp + my_net.fc1.bias.data.numpy()
test_comp = (abs(test_comp) + test_comp)/2
test_comp = np.matmul(my_net.fc2.weight.data.numpy(), test_comp)
test_comp = test_comp + my_net.fc2.bias.data.numpy()
test_comp = (abs(test_comp) + test_comp)/2
test_comp = torch.from_numpy(np.matmul(my_net.fc3.weight.data.numpy(), test_comp))
test_comp = test_comp + my_net.fc3.bias.data
test_comp = F.log_softmax(test_comp)
print(test_comp)

tensor([-1.3707e+01, -2.0329e+01, -1.3905e+01, -1.0911e+01, -1.8184e+01,
        -1.1276e+01, -2.7975e+01, -3.4332e-05, -1.4850e+01, -1.3861e+01])


C:\Users\ethan\AppData\Local\Temp\ipykernel_4144\2749662064.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  test_comp = F.log_softmax(test_comp)
